<center>
<h2>DV2599 - PROJECT</h2>
<h3>Predicting Payment date</h3>
<h4>
Rasmus Dunder - radn21@student.bth.se
</h4>
<h4>
Rasmus Johansson - rajo22@student.bth.se
</h4>
</center>

#### Import modules / libraries

In [2]:
import pandas as pd
import datetime
import math

#### Data cleaning

In [3]:
#Importing the dataset
df = pd.read_csv('dataset.csv')

#Dropping not needed columns
df = df.drop(columns=['name_customer', 'buisness_year', 'invoice_currency', 'doc_id','posting_id', 'area_business','document type', 'baseline_create_date', 'document_create_date.1', 'document_create_date'])

# df['invoice_id'] = df['invoice_id'].astype(str)
# Print ratio of open/closed invoices
not_yetpayed = df['isOpen'].value_counts().sort_index()
quality_ratios = round((not_yetpayed / not_yetpayed.sum()),4) * 100
print(f"Ratios: {quality_ratios}")
print(f"Counts: {not_yetpayed}")

# Drop all Columns where invoice is not yet paid
df = df[df['isOpen'] == 0]

#Printing the nullvalues in the dataset
print(f"\nNull values in the dataset: {sum(df.isnull().sum(axis=1))}\n")

#Dropping the nullvalues
df.dropna(inplace=True)

#Dropping the duplicates from the dataset
df.drop_duplicates(subset=['invoice_id'], inplace=True)

#Removing now not needed columns
df = df.drop(columns=['invoice_id', 'isOpen'])

#Making all dates the same type
df['due_in_date'] = pd.to_datetime(df['due_in_date'], format='%Y%m%d')
df['clear_date'] = pd.to_datetime(df['clear_date'],  errors='coerce')
df['posting_date'] = pd.to_datetime(df['posting_date'], format='%m/%d/%Y')

# Drop Columns where due_in_date is before posting date
df = df[df['due_in_date'] >= df['posting_date']]

#Creating 3 new columns
df['payment_term'] = (df['due_in_date'] - df['posting_date']).dt.days
df['delayed_payment_days'] = (df['clear_date'] - df['due_in_date']).dt.days
df['payed_on_time'] = df['delayed_payment_days'] <= 0

print(df.iloc[0:2])

Ratios: isOpen
0    80.0
1    20.0
Name: count, dtype: float64
Counts: isOpen
0    40000
1    10000
Name: count, dtype: int64

Null values in the dataset: 6

  business_code cust_number clear_date posting_date due_in_date  \
0          U001   200769623 2020-02-11   2020-01-26  2020-02-10   
1          U001   200980828 2019-08-08   2019-07-22  2019-08-11   

   total_open_amount cust_payment_terms  payment_term  delayed_payment_days  \
0           54273.28               NAH4            15                     1   
1           79656.60               NAD1            20                    -3   

   payed_on_time  
0          False  
1           True  


Att tänka på

- Outliers
- Normalisering

In [16]:
from scipy import stats

#Checking for ratio on True/False
payed_time = df['payed_on_time'].value_counts().sort_index()
quality_ratios = round((payed_time / payed_time.sum()),4) * 100
print(f"Ratios: {quality_ratios}")
print(f"Counts: {payed_time}")

#Checking for ratio on days
payed_time = df['delayed_payment_days'].value_counts().sort_index()
quality_ratios = round((payed_time / payed_time.sum()),4) * 100
print(f"Ratios: {quality_ratios}")
print(f"Counts: {payed_time}")

# Checking for outliers
df_zscore = df.copy()
df_zscore['Z-score']= stats.zscore(df_zscore['delayed_payment_days']) #Calculating z-score for Value
threshold = 3
df_icu_zscore = df_zscore [abs(df_zscore ['Z-score']) > threshold]
print("Number of outliers on Value", df_icu_zscore['delayed_payment_days'].count())
zscore_min = abs(df_icu_zscore['delayed_payment_days']).min()
zscore_max = abs(df_icu_zscore['delayed_payment_days']).max()
print(f"Outlier value range: Min = {zscore_min}, Max = {zscore_max}")
print(list(df.columns))

df_payement_days = df.copy()
df_payement_ontime = df.copy()

df_payement_days = df_payement_days.drop(columns=['payed_on_time'])
df_payement_ontime = df_payement_ontime.drop(columns=['delayed_payment_days'])

Ratios: payed_on_time
False    41.76
True     58.24
Name: count, dtype: float64
Counts: payed_on_time
False    16308
True     22740
Name: count, dtype: int64
Ratios: delayed_payment_days
-89     0.00
-87     0.00
-79     0.00
-75     0.01
-74     0.01
        ... 
 157    0.00
 171    0.00
 195    0.00
 202    0.00
 204    0.00
Name: count, Length: 175, dtype: float64
Counts: delayed_payment_days
-89     1
-87     1
-79     1
-75     5
-74     5
       ..
 157    1
 171    1
 195    1
 202    1
 204    1
Name: count, Length: 175, dtype: int64
Number of outliers on Value 1018
Outlier value range: Min = 32, Max = 204
['business_code', 'cust_number', 'clear_date', 'posting_date', 'due_in_date', 'total_open_amount', 'cust_payment_terms', 'payment_term', 'delayed_payment_days', 'payed_on_time']
